[View in Colaboratory](https://colab.research.google.com/github/ami432/INKERS/blob/master/Bharatkumar_Batch3_Assignment4B.ipynb)

In [2]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 20
l = 40
num_filter = 32
compression = 0.5
dropout_rate = 0.3

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
# Dense Block
def add_denseblock(input, num_filter = 32, dropout_rate = 0.3):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 32, dropout_rate = 0.3):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 32
dropout_rate = 0.3
l = 20
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [11]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 32)   128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 32)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

concatenate_52 (Concatenate)    (None, 8, 8, 208)    0           concatenate_51[0][0]             
                                                                 dropout_54[0][0]                 
__________________________________________________________________________________________________
batch_normalization_55 (BatchNo (None, 8, 8, 208)    832         concatenate_52[0][0]             
__________________________________________________________________________________________________
activation_55 (Activation)      (None, 8, 8, 208)    0           batch_normalization_55[0][0]     
__________________________________________________________________________________________________
conv2d_56 (Conv2D)              (None, 8, 8, 16)     29952       activation_55[0][0]              
__________________________________________________________________________________________________
dropout_55 (Dropout)            (None, 8, 8, 16)     0           conv2d_56[0][0]                  
__________

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [13]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 661s 13ms/step - loss: 1.5607 - acc: 0.4184 - val_loss: 1.4883 - val_acc: 0.4846
Epoch 2/20
19072/50000 [==========>...................] - ETA: 6:04 - loss: 1.2255 - acc: 0.5520

50000/50000 [==============================] - 633s 13ms/step - loss: 1.1505 - acc: 0.5811 - val_loss: 1.6177 - val_acc: 0.5324
Epoch 3/20
45184/50000 [==========================>...] - ETA: 56s - loss: 0.9800 - acc: 0.6485

50000/50000 [==============================] - 633s 13ms/step - loss: 0.9756 - acc: 0.6508 - val_loss: 2.8081 - val_acc: 0.4286
Epoch 4/20
50000/50000 [==============================] - 633s 13ms/step - loss: 0.8545 - acc: 0.6970 - val_loss: 1.1787 - val_acc: 0.6590
Epoch 5/20
 2944/50000 [>.............................] - ETA: 9:14 - loss: 0.7544 - acc: 0.7225

50000/50000 [==============================] - 633s 13ms/step - loss: 0.7659 - acc: 0.7275 - val_loss: 1.2449 - val_acc: 0.6374
Epoch 6/20
39040/50000 [======================>.......] - ETA: 2:09 - loss: 0.7062 - acc: 0.7485

50000/50000 [==============================] - 633s 13ms/step - loss: 0.7004 - acc: 0.7521 - val_loss: 1.5442 - val_acc: 0.6288
Epoch 7/20
50000/50000 [==============================] - 633s 13ms/step - loss: 0.6485 - acc: 0.7721 - val_loss: 1.0868 - val_acc: 0.7091
Epoch 8/20
 1536/50000 [..............................] - ETA: 9:32 - loss: 0.5946 - acc: 0.7852

...

50000/50000 [==============================] - 632s 13ms/step - loss: 0.4298 - acc: 0.8500 - val_loss: 0.9421 - val_acc: 0.7768
Epoch 15/20
38272/50000 [=====================>........] - ETA: 2:17 - loss: 0.4151 - acc: 0.8561

50000/50000 [==============================] - 632s 13ms/step - loss: 0.4196 - acc: 0.8539 - val_loss: 1.1039 - val_acc: 0.7322
Epoch 16/20
50000/50000 [==============================] - 631s 13ms/step - loss: 0.3979 - acc: 0.8622 - val_loss: 1.0201 - val_acc: 0.7427
Epoch 17/20
 1280/50000 [..............................] - ETA: 9:34 - loss: 0.3524 - acc: 0.8727

50000/50000 [==============================] - 632s 13ms/step - loss: 0.3840 - acc: 0.8660 - val_loss: 0.7002 - val_acc: 0.8148
Epoch 18/20
38272/50000 [=====================>........] - ETA: 2:17 - loss: 0.3645 - acc: 0.8712

50000/50000 [==============================] - 632s 13ms/step - loss: 0.3680 - acc: 0.8705 - val_loss: 0.6560 - val_acc: 0.8171
Epoch 19/20
50000/50000 [==============================] - 633s 13ms/step - loss: 0.3553 - acc: 0.8760 - val_loss: 0.9100 - val_acc: 0.7768
Epoch 20/20
 1280/50000 [..............................] - ETA: 9:37 - loss: 0.3112 - acc: 0.8922

50000/50000 [==============================] - 633s 13ms/step - loss: 0.3411 - acc: 0.8804 - val_loss: 0.7953 - val_acc: 0.7899


In [14]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 55s 5ms/step
Test loss: 0.7952847223520279
Test accuracy: 0.7899


In [15]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('DNST_model.h5')